In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
from Bio import SeqIO
import inspect

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
FASTA_PATH = pathlib.Path(DATA_PATH).joinpath('uniprot-compressed_true_download_true_format_fasta_includeIsoform_tr-2023.02.17-22.00.02.49.fasta')
RESULT_PATH = DATA_PATH.joinpath('gene_symbol_protein_sequences.pkl')


In [2]:
records = []
for data in SeqIO.parse(FASTA_PATH, "fasta"):
    record={}
    for index, value in enumerate(re.findall(r'(?:(?<=^)[^|]+|(?<=\|)[^| ]+|(?<=\|)[^ ]+)|(?<=\ ).+?(?= \w\w\=|$)', data.description)):
        value = value.strip()
        if index == 0:
            record['db'] = value
        elif index == 1:
            record['UniqueIdentifier'] = value
        elif index == 2:
            record['EntryName'] = value
        elif index == 3:
            record['ProteinName'] = value
        else:
            k, v = re.split(r'\=', value)
            record[k] = v

    record['seq'] = str(data.seq)
    
    records.append(record)

In [3]:
df = pd.DataFrame(records)

df.to_pickle(RESULT_PATH)

In [5]:
df = pd.read_pickle(RESULT_PATH)

df.loc[df['UniqueIdentifier'].duplicated()]

,db,UniqueIdentifier,EntryName,ProteinName,OS,OX,GN,PE,SV,seq
